# Join under inequality condition

## Add campaign information to reservation data

### Pandas

In [1]:
import pandas as pd

In [2]:
pd_reservation = pd.read_parquet("../../data/reservation.parquet")
pd_campaign = pd.read_parquet("../../data/campaign.parquet")

In [3]:
pd_reservation.head()

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
0,1,2460,53431,2013-12-31 07:00:14+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,2,37800,reserved,NaT
1,2,962,488390,2013-12-31 08:23:35+09:00,2014-12-31 00:00:00+09:00,2015-01-02 00:00:00+09:00,2,3,42000,reserved,NaT
2,3,558,341335,2013-12-31 09:02:05+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,2,20400,reserved,NaT
3,4,3666,398981,2013-12-31 23:44:54+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,4,39600,reserved,NaT
4,5,2180,220381,2014-01-01 02:47:50+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,3,16500,reserved,NaT


In [4]:
pd_campaign.head()

,campaign_name,starts_at,ends_at
0,初売りキャンペーン,2015-01-01 00:00:00+09:00,2015-01-14 23:59:59+09:00
1,春休みキャンペーン,2015-02-01 00:00:00+09:00,2015-03-20 23:59:59+09:00
2,GWキャンペーン,2015-04-01 00:00:00+09:00,2015-04-30 23:59:59+09:00
3,夏休みキャンペーン,2015-07-01 00:00:00+09:00,2015-08-20 23:59:59+09:00
4,オータムキャンペーン,2015-09-01 00:00:00+09:00,2015-10-20 23:59:59+09:00


In [5]:
(
    pd_reservation
    .assign(reserve_date=lambda df: df.reserved_at.dt.normalize())
    .merge(
        pd_campaign
        .assign(reserve_date=lambda df:
               df.apply(lambda r: pd.date_range(r.starts_at, r.ends_at), axis=1))
        [["campaign_name", "reserve_date"]]
        .explode("reserve_date"),
        on="reserve_date", how="left"
    )
    .drop(columns="reserve_date")
)

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at,campaign_name
0,1,2460,53431,2013-12-31 07:00:14+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,2,37800,reserved,NaT,NaN
1,2,962,488390,2013-12-31 08:23:35+09:00,2014-12-31 00:00:00+09:00,2015-01-02 00:00:00+09:00,2,3,42000,reserved,NaT,NaN
2,3,558,341335,2013-12-31 09:02:05+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,2,20400,reserved,NaT,NaN
3,4,3666,398981,2013-12-31 23:44:54+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,4,39600,reserved,NaT,NaN
4,5,2180,220381,2014-01-01 02:47:50+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,3,16500,reserved,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,1999996,2357,280303,2019-12-27 12:14:07+09:00,2019-12-29 00:00:00+09:00,2019-12-30 00:00:00+09:00,1,4,26000,reserved,NaT,冬休みキャンペーン
1999996,1999997,319,499387,2019-12-27 14:08:57+09:00,2019-12-29 00:00:00+09:00,2019-12-30 00:00:00+09:00,1,1,7100,canceled,2019-12-28 21:37:30+09:00,冬休みキャンペーン
1999997,1999998,2834,461799,2019-12-28 04:59:51+09:00,2019-12-30 00:00:00+09:00,2019-12-31 00:00:00+09:00,1,5,105000,reserved,NaT,冬休みキャンペーン
1999998,1999999,3643,163568,2019-12-28 11:56:19+09:00,2019-12-30 00:00:00+09:00,2019-12-31 00:00:00+09:00,1,1,17000,reserved,NaT,冬休みキャンペーン


### Polars

In [6]:
import polars as pl

In [7]:
pl_reservation = pl.read_parquet("../../data/reservation.parquet")
pl_campaign    = pl.read_parquet("../../data/campaign.parquet")

In [8]:
pl_reservation.head()

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]"
1,2460,53431,2013-12-31 07:00:14 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,2,37800,"""reserved""",null
2,962,488390,2013-12-31 08:23:35 JST,2014-12-31 00:00:00 JST,2015-01-02 00:00:00 JST,2,3,42000,"""reserved""",null
3,558,341335,2013-12-31 09:02:05 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,2,20400,"""reserved""",null
4,3666,398981,2013-12-31 23:44:54 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,4,39600,"""reserved""",null
5,2180,220381,2014-01-01 02:47:50 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,3,16500,"""reserved""",null


In [9]:
pl_campaign.head()

campaign_name,starts_at,ends_at
str,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]"
"""初売りキャンペーン""",2015-01-01 00:00:00 JST,2015-01-14 23:59:59 JST
"""春休みキャンペーン""",2015-02-01 00:00:00 JST,2015-03-20 23:59:59 JST
"""GWキャンペーン""",2015-04-01 00:00:00 JST,2015-04-30 23:59:59 JST
"""夏休みキャンペーン""",2015-07-01 00:00:00 JST,2015-08-20 23:59:59 JST
"""オータムキャンペーン""",2015-09-01 00:00:00 JST,2015-10-20 23:59:59 JST


In [10]:
(
    pl_reservation
    .with_columns(reserve_date=pl.col("reserved_at").dt.date())
    .join(
        pl_campaign
        .select([
            pl.date_ranges(
                pl.col("starts_at").dt.date(), pl.col("ends_at").dt.date(), "1d"
            )
            .alias("reserve_date"),
            pl.col("campaign_name")
        ])
        .explode("reserve_date"),
        on="reserve_date",
        how="left"
    )
    .drop("reserve_date")
)

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at,campaign_name
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]",str
1,2460,53431,2013-12-31 07:00:14 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,2,37800,"""reserved""",null,null
2,962,488390,2013-12-31 08:23:35 JST,2014-12-31 00:00:00 JST,2015-01-02 00:00:00 JST,2,3,42000,"""reserved""",null,null
3,558,341335,2013-12-31 09:02:05 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,2,20400,"""reserved""",null,null
4,3666,398981,2013-12-31 23:44:54 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,4,39600,"""reserved""",null,null
5,2180,220381,2014-01-01 02:47:50 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,3,16500,"""reserved""",null,null
…,…,…,…,…,…,…,…,…,…,…,…
1999996,2357,280303,2019-12-27 12:14:07 JST,2019-12-29 00:00:00 JST,2019-12-30 00:00:00 JST,1,4,26000,"""reserved""",null,"""冬休みキャンペーン"""
1999997,319,499387,2019-12-27 14:08:57 JST,2019-12-29 00:00:00 JST,2019-12-30 00:00:00 JST,1,1,7100,"""canceled""",2019-12-28 21:37:30 JST,"""冬休みキャンペーン"""
1999998,2834,461799,2019-12-28 04:59:51 JST,2019-12-30 00:00:00 JST,2019-12-31 00:00:00 JST,1,5,105000,"""reserved""",null,"""冬休みキャンペーン"""
